# API walking

This notebook enumerates _TruLens_ components that are part of the high-level
and low-level APIs. It is used to develop and debug API compatibility tests.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.resolve()))

In [ ]:
import inspect
from typing import Any, Dict, List, Optional, Set, Union

import pydantic
import yaml

from tests.utils import get_class_members
from tests.utils import get_module_members
from tests.utils import get_module_names
import trulens_eval

In [ ]:
# Check class members:
mems = get_class_members(trulens_eval.schema.feedback.FeedbackResult)
mems.api_lows

In [ ]:
def _prettysig(func):
    sig = inspect.signature(func)
    name = func.__name__

    ret = "def " + name + "(\n"

    ret += str(sig) + "\n"

    first = True
    posonly = True
    kwonly = False

    for name, param in sig.parameters.items():
        if posonly and param.kind is not inspect.Parameter.POSITIONAL_ONLY:
            posonly = False
            if not first:
                ret += "\t/,\n"

        elif not kwonly and param.kind is inspect.Parameter.KEYWORD_ONLY:
            kwonly = True
            ret += "\t*,\n"

        if param.kind is inspect.Parameter.VAR_POSITIONAL:
            ret += f"\t*{name}"
        elif param.kind is inspect.Parameter.VAR_KEYWORD:
            ret += f"\t**{name}"
        else:
            ret += f"\t{name},\n"

        if first:
            first = False

    ret += "\n): pass"

    return ret


def _strattr(val, name: str):
    attr_val = inspect.getattr_static(val, name, None)
    if attr_val is None:
        return None

    if isinstance(attr_val, str):
        return attr_val

    if hasattr(attr_val, "__get__"):
        return attr_val.__get__(val)

    if hasattr(attr_val, "__name__"):
        return attr_val.__name__

    if inspect.ismethoddescriptor(attr_val):
        return "methoddescriptor"

    if inspect.isgetsetdescriptor(attr_val):
        return attr_val.name
        return "getsetdescriptor"

    if inspect.ismemberdescriptor(attr_val):
        return "memberdescriptor"

    if inspect.isdatadescriptor(attr_val):
        return "datadescriptor"

    return str(dir(attr_val))


def _qualname(val):
    ret = ""

    module = _strattr(val, "__module__")
    if module is not None:
        ret += module + "."

    qualname = _strattr(val, "__qualname__")
    name = _strattr(val, "__name__")

    if qualname is not None:
        ret += qualname
    elif name is not None:
        ret += name
    else:
        ret += "?"

    return ret


class Ident(pydantic.BaseModel):
    qualname: str
    name: str

    @pydantic.model_validator(mode="wrap")
    @classmethod
    def validate_from_qualname(cls, value, handler):
        if isinstance(value, str):
            _, name = value.rsplit(".", 1)
            return Ident(qualname=value, name=name)
        else:
            return handler(value)

    @pydantic.model_serializer
    def dump_as_string(self, **kwargs):
        return str(self.qualname)

    @staticmethod
    def of_value(val: Any) -> "Ident":
        name = _strattr(val, "__name__")
        if name is not None:
            return Ident(qualname=_qualname(val), name=name)

        return Ident(qualname=_qualname(val), name="?")

    @staticmethod
    def of_value_type(val: Any) -> "Ident":
        typ = type(val)
        if isinstance(typ, type):
            return Ident(qualname=_qualname(typ), name=typ.__name__)

        raise ValueError(f"Can't get type of {val}")


class Member(pydantic.BaseModel):
    module_ident: Ident
    member_ident: Ident

    @staticmethod
    def of_value(mod, val):
        module_ident = Ident.of_value(mod)
        member_ident = Ident.of_value(val)

        if inspect.ismethod(val):
            return Method(
                module_ident=module_ident,
                member_ident=member_ident,
                sig=Signature.of_sig(inspect.signature(val)),
                class_ident=Ident.of_value(val.__self__),
                type_ident=Ident.of_value_type(val),
            )

        if inspect.isfunction(val):
            return Function(
                module_ident=module_ident,
                member_ident=member_ident,
                sig=Signature.of_sig(inspect.signature(val)),
                type_ident=Ident.of_value_type(val),
            )

        if inspect.ismodule(val):
            return Module(
                module_ident=module_ident,
                member_ident=member_ident,
                name=Ident.of_value(val),
                type_ident=Ident.of_value_type(val),
            )

        if inspect.isclass(val):
            return Class(
                module_ident=module_ident,
                member_ident=member_ident,
                base_idents=[Ident.of_value(b) for b in val.__bases__],
                init_sig=Signature.of_sig(inspect.signature(val.__init__)),
                type_ident=Ident.of_value_type(val),
            )

        if isinstance(val, type):
            return Member(
                module_ident=module_ident,
                member_ident=Ident.of_value(val),
            )


class Value(Member):
    type_ident: Ident


class Module(Value):
    name: Ident


class Const(Value):
    value: str

    @staticmethod
    def of_value(val):
        return Const(
            value=str(val),
            type_ident=Ident.of_value_type(type(val)),
            module_ident=Ident.of_value(val),
            member_ident=Ident.of_value(val),
        )


class Parameter(pydantic.BaseModel):
    name: str
    default: Optional[Const]
    kwarg_only: bool
    pos_only: bool
    type_ident: Optional[Ident]

    @staticmethod
    def of_param(param):
        return Parameter(
            name=param.name,
            default=Const.of_value(param.default)
            if param.default is not inspect.Parameter.empty
            else None,
            kwarg_only=param.kind == inspect.Parameter.KEYWORD_ONLY,
            pos_only=param.kind == inspect.Parameter.POSITIONAL_ONLY,
            type_ident=Ident.of_value_type(param.annotation)
            if param.annotation is not inspect.Parameter.empty
            else None,
        )


class Signature(pydantic.BaseModel):
    param_annot_idents: Dict[str, Parameter]  # types are idents
    ret_annot_ident: Ident

    @staticmethod
    def of_sig(sig):
        return Signature(
            param_annot_idents={
                p: Parameter.of_param(pv) for p, pv in sig.parameters.items()
            },
            ret_annot_ident=Ident.of_value_type(sig.return_annotation),
        )


class Class(Value):
    base_idents: List[Ident]
    init_sig: Signature


class Function(Value):
    sig: Signature


class Method(Function):
    class_ident: Ident


def serialize_stub(
    mod, val, done: Optional[Set[int]] = None
) -> Union[str, Dict]:
    #    print(inspect.getmodule(typ))

    # if not isinstance(val, type):
    #    return {"typ": str(type(val))}

    if done is None:
        done = set()

    return Member.of_value(mod, val)

In [ ]:
content = dict()
for modname in get_module_names(
    Path(trulens_eval.__file__).parent.parent, matching="trulens_eval"
):
    mod = get_module_members(modname)
    if mod is None:
        continue

    print(modname)

    highs = {}
    lows = {}
    module_content = {"highs": highs, "lows": lows}
    content[modname] = module_content

    for e in mod.api_highs:
        # print(f"  [hih] {e.qualname}: {e.typ}")
        s = serialize_stub(e.mod, e.val)
        if s is not None:
            highs[e.qualname] = s.model_dump()

    for e in mod.api_lows:
        # print(f"  [low] {e.qualname}: {e.typ}")
        s = serialize_stub(e.mod, e.val)
        if s is not None:
            lows[e.qualname] = s.model_dump()

In [ ]:
print(yaml.dump(content))